In [1]:
import pandas as pd

import os
import sys
from dotenv import load_dotenv
load_dotenv()
import sys
sys.path.insert(1, os.getenv('LIBRARY_PATH'))
from datetime import date
from dateutil.relativedelta import relativedelta
import scrapper
import data_pipeline
from sqlalchemy import create_engine
def clean_string(string):
    return string.replace(" ", "_").replace(",","").replace("(","").replace(")","").replace("-","_").replace("__","_").lower()[:60]

def upload_sql(query, data, table_name, latest_month, pick_member_1, pick_member_2, x, y, transpose):
    conn_string = 'mysql+pymysql://' + os.environ["USERNAME"] + ':' + os.environ["PASSWORD"] + '@' + os.environ["HOST"] + '/' + os.environ["DATABASE"] 
    engine = create_engine(conn_string, connect_args={
        "ssl": {
            "ssl_ca": "ca.pem",
            "ssl_cert": "client-cert.pem",
            "ssl_key": "client-key.pem"
        }
    })
    data.to_sql(table_name, engine, if_exists='replace', index=False)
    if transpose == False:
        print("Inserted data from " + x + " and " + y + " into " + table_name + " successfully")
    else:
        print("Inserted data into " + table_name + " successfully")


In [2]:
industries_dict = {'3.1': 'industrial_aggregate_including_unclassified_businesses', '3.2': 'industrial_aggregate_excluding_unclassified_businesses', '3.3': 'goods_producing_industries', '3.4': 'forestry_logging_and_support', '3.10': 'mining_quarrying_and_oil_and_gas_extraction', '3.17': 'utilities', '3.21': 'construction', '3.34': 'manufacturing', '3.143': 'service_producing_industries', '3.144': 'trade', '3.215': 'transportation_and_warehousing', '3.253': 'information_and_cultural_industries', '3.267': 'finance_and_insurance', '3.284': 'real_estate_and_rental_and_leasing', '3.295': 'professional_scientific_and_technical_services', '3.306': 'management_of_companies_and_enterprises', '3.321': 'educational_services', '3.331': 'health_care_and_social_assistance', '3.354': 'arts_entertainment_and_recreation', '3.367': 'accommodation_and_food_services', '3.377': 'other_services_except_public_administration', '3.393': 'public_administration'}
pick_members_2 = {"names": ['2.2'], "values": ['average_weekly_earnings']}
pick_members_3 = {"names": ['3.1', '3.2', '3.3', '3.4', '3.10', '3.17', '3.21', '3.34', '3.143', '3.144', '3.215', '3.253', '3.267', '3.284', '3.295', '3.306', '3.321', '3.331', '3.354', '3.367', '3.377', '3.393'],"values": ['industrial_aggregate_including_unclassified_businesses', 'industrial_aggregate_excluding_unclassified_businesses', 'goods_producing_industries', 'forestry_logging_and_support', 'mining_quarrying_and_oil_and_gas_extraction', 'utilities', 'construction', 'manufacturing', 'service_producing_industries', 'trade', 'transportation_and_warehousing', 'information_and_cultural_industries', 'finance_and_insurance', 'real_estate_and_rental_and_leasing', 'professional_scientific_and_technical_services', 'management_of_companies_and_enterprises', 'educational_services', 'health_care_and_social_assistance', 'arts_entertainment_and_recreation', 'accommodation_and_food_services', 'other_services_except_public_administration', 'public_administration'] }
pick_members_dict = {"&pickMembers%5B0%5D=": pick_members_2, "&pickMembers%5B1%5D=": pick_members_3}
filter_names = {"pick_member_1": "Estimate", "pick_member_2": "North American Industry Classification System (NAICS)"}
table_name = "average_weekly_earnings_by_industry" 

In [3]:
pid="1410022301"
table_name = "average_weekly_earnings_by_industry"
pick_members_dict = {"&pickMembers%5B0%5D=": pick_members_2, "&pickMembers%5B1%5D=": pick_members_3}
filter_names = {"pick_member_1": "Estimate", "pick_member_2": "North American Industry Classification System (NAICS)"}
transpose = True

In [4]:
start_month="01"
start_year="2022"
end_month="12"
end_year="2022"
referencePeriods="20220101%2C20221001"

referencePeriods = start_year + start_month + "01" + "%2C" + end_year + end_month + "28" 
keys = list(pick_members_dict.keys())
values = list((pick_members_dict[key]["names"] for key in keys))

filter_names_keys = list(filter_names.keys())
filter_names_values = list(filter_names.values())
pick_members_1_dict = dict(zip(pick_members_dict["&pickMembers%5B0%5D="]["names"], pick_members_dict["&pickMembers%5B0%5D="]["values"]))
pick_members_2_dict = dict(zip(pick_members_dict["&pickMembers%5B1%5D="]["names"], pick_members_dict["&pickMembers%5B1%5D="]["values"]))

conn_string = 'mysql+pymysql://' + os.environ["USERNAME"] + ':' + os.environ["PASSWORD"] + '@' + os.environ["HOST"] + '/' + os.environ["DATABASE"] 
engine = create_engine(conn_string, connect_args={
    "ssl": {
        "ssl_ca": "ca.pem",
        "ssl_cert": "client-cert.pem",
        "ssl_key": "client-key.pem"
    }
})

In [5]:
if transpose:
    final_df = pd.DataFrame()
result = [(x, y) for x in values[0] for y in values[1]]

In [6]:
for x, y in result:
    pick_member_1=pick_members_1_dict[x]
    pick_member_2=pick_members_2_dict[y]
    url = 'https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=' + pid + '&pickMembers%5B0%5D='+ x + '&pickMembers%5B1%5D='+ y + '&cubeTimeFrame.startMonth='+ start_month + '&cubeTimeFrame.startYear=' + start_year + '&cubeTimeFrame.endMonth=' + end_month +'&cubeTimeFrame.endYear=' + end_year + '&referencePeriods=' + referencePeriods
    df = scrapper.simple_scrapper(url, filter_names.values())

    if transpose:
        new_df = df.melt(['date', 'month'], var_name='province', value_name=pick_member_2)
        provinces = ['canada', 'newfoundland_and_labrador', 'prince_edward_island', 'nova_scotia', 'new_brunswick', 'quebec', 'ontario', 'manitoba', 'saskatchewan', 'alberta', 'british_columbia', 'yukon', 'northwest_territories', 'nunavut']
        new_df = new_df[new_df['province'].isin(provinces)]
        if final_df.empty:
            final_df["month"] = new_df["month"]
            final_df["date"] = new_df["date"]
            final_df["province"] = new_df["province"]

        final_df[pick_member_2] = new_df[pick_member_2]
    
    if transpose == False:
        df.to_sql(table_name, con=engine, if_exists='replace', index=False)
        print("Inserted data from " + x + " and " + y + " into " + table_name + " successfully")

if transpose == True:
    final_df.to_sql(table_name, engine, if_exists='replace', index=False)
    print("Table " + table_name + " created successfully")


Table average_weekly_earnings_by_industry created successfully


In [7]:
url = 'https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410022301&pickMembers%5B0%5D=2.2&pickMembers%5B1%5D=3.3&cubeTimeFrame.startMonth=01&cubeTimeFrame.startYear=2022&cubeTimeFrame.endMonth=12&cubeTimeFrame.endYear=2022&referencePeriods=20220101%2C20221201'
test = scrapper.simple_scrapper(url, filter_names.values())

In [8]:
provinces = ['canada', 'newfoundland_and_labrador', 'prince_edward_island', 'nova_scotia', 'new_brunswick', 'quebec', 'ontario', 'manitoba', 'saskatchewan', 'alberta', 'british_columbia', 'yukon', 'northwest_territories', 'nunavut']

new_test = test.melt(['date', 'month'], var_name='province', value_name='type_of_employee')
new_test = new_test[new_test['province'].isin(provinces)]
